In [ ]:
!pip uninstall keras-nightly

!pip uninstall -y tensorflow

!pip install h5py==2.10.0  

In [ ]:
!pip install tensorflow==1.15
!pip install keras==2.2.4

In [ ]:
import os
import keras
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report, accuracy_score
import itertools
from keras import callbacks
from keras.utils.vis_utils import plot_model
from keras.models import Model
from capsulelayers import Length, Mask,CapsuleLayer, PrimaryCap
from keras.models import load_model
from keras.callbacks import TensorBoard
import keras.backend as K
import tensorflow as tf
from keras import layers, models
from keras import initializers, layers
from __future__ import print_function
from keras.datasets import mnist
from keras.models import Model
from keras.layers import Input, Dense, TimeDistributed
from keras.layers import Activation, Dense, Input
from keras.layers import LSTM
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten,BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Reshape, Conv2DTranspose
from keras.utils import CustomObjectScope
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from keras.utils import np_utils
from keras.layers import Dense, Dropout, GaussianNoise, Conv1D
import seaborn as sns
from time import time
from keras import layers, models, optimizers
from keras import backend as K
import cv2
import itertools
from sklearn import svm, datasets
import sklearn.model_selection as model_selection
from sklearn.feature_selection import RFE
from sklearn.metrics import f1_score
from tqdm import tqdm
import random
import matplotlib.pyplot as plt
from utils import combine_images
from PIL import Image
from sklearn.model_selection import train_test_split
from capsulelayers import CapsuleLayer, PrimaryCap, Length, Mask
from keras.layers import concatenate
from tensorflow import keras
import pickle

/content/drive/My Drive/AI/CapsNet-Keras


Using TensorFlow backend.


In [ ]:
%cd /content/drive/My Drive/...dataset_path
data_train = pd.read_csv('sign_mnist_train.csv')
X_full_tr = data_train.iloc[:,1:]
y_full_tr= data_train.iloc[:,:1]
x_train=X_full_tr 
y_train=y_full_tr
data_test = pd.read_csv('sign_mnist_test.csv')
X_full_ts = data_test.iloc[:,1:]
y_full_ts= data_test.iloc[:,:1]
x_test=X_full_ts
y_test=y_full_ts


x_train = x_train.values.reshape(-1, 28, 28, 1).astype('float32') / 255.
x_test = x_test.values.reshape(-1, 28, 28, 1).astype('float32') / 255.
y_train = to_categorical(y_train.astype('float32'))
y_test = to_categorical(y_test.astype('float32'))
print('Training', x_train.shape, x_train.max())
print('Testing', x_test.shape, x_test.max())
print('Training', y_train.shape, y_train.max())
print('Testing', y_test.shape, y_test.max())

In [ ]:
#capsNet_model 
def CapsNet(input_shape, n_class, num_routing):
    """
    A Capsule Network on MNIST.
    :param input_shape: data shape, 4d, [None, width, height, channels]
    :param n_class: number of classes
    :param num_routing: number of routing iterations
    :return: A Keras Model with 2 inputs and 2 outputs
    """
    x = layers.Input(shape=input_shape)

    # Layer 1: Just a conventional Conv2D layer
    conv1 = layers.Conv2D(filters=256, kernel_size=9, strides=1, padding='valid', activation='relu', name='conv1')(x)
    # Layer 2: Conv2D layer with `squash` activation, then reshape to [None, num_capsule, dim_vector]
    primarycaps = PrimaryCap(conv1, dim_vector=8, n_channels=16, kernel_size=9, strides=2, padding='valid')
    # Layer 3: Capsule layer. Routing algorithm works here.
    digitcaps = CapsuleLayer(num_capsule=n_class, dim_vector=16, num_routing=num_routing, name='digitcaps')(primarycaps)

    # Layer 4: This is an auxiliary layer to replace each capsule with its length. Just to match the true label's shape.
    # If using tensorflow, this will not be necessary. :)
    out_caps = Length(name='out_caps')(digitcaps)

    # Decoder network.
    y = layers.Input(shape=(n_class,))
    masked = Mask()([digitcaps, y])  # The true label is used to mask the output of capsule layer.
    x_recon = layers.Dense(64, activation='relu')(masked)
    x_recon = layers.Dense(128, activation='relu')(x_recon)
    x_recon = layers.Dense(1600, activation='softmax')(x_recon)
    x_recon = layers.Reshape(target_shape=input_shape, name='out_recon')(x_recon)

    # two-input-two-output keras Model
    return models.Model([x, y], [out_caps, x_recon])
"----------------------------------------------------------------------------------------------"
def margin_loss(y_true, y_pred):
    """
    Margin loss for Eq.(4). When y_true[i, :] contains not just one `1`, this loss should work too. Not test it.
    :param y_true: [None, n_classes]
    :param y_pred: [None, num_capsule]
    :return: a scalar loss value.
    """
    L = y_true * K.square(K.maximum(0., 0.9 - y_pred)) + \
        0.25 * (1 - y_true) * K.square(K.maximum(0., y_pred - 0.1))

    return K.mean(K.sum(L, 1))
"------------------------------------------------------------------------------------------------"
model = CapsNet(input_shape=[28, 28, 1],
                n_class=25,
                num_routing=3)
model.summary()
try:
    plot_model(model, to_file='model.png', show_shapes=True)
except Exception as e:
    print('No fancy plot {}'.format(e))
 

In [ ]:
# unpacking the data
    #(x_train, y_train), (x_test, y_test) = data

    # callbacks
log = callbacks.CSVLogger('log.csv')
    #checkpoint = callbacks.ModelCheckpoint('weights-{epoch:02d}.h5',
     #                                      save_best_only=True, save_weights_only=True, verbose=1)
#tb = callbacks.TensorBoard(log_dir="/content/drive/My Drive/AI/CapsNet-Keras/logs/{}",
 #                           batch_size=128, histogram_freq=1)
lr_decay = callbacks.LearningRateScheduler(schedule=lambda epoch: 0.001 * np.exp(-epoch / 10.))
# compile the mode
model.compile(optimizer=optimizers.Adam(lr=0.001),
                  loss=[margin_loss, 'mse'],
                  loss_weights=[1., 0.0392],
                  metrics={'out_caps': 'accuracy'})
                  
                  
    

# Training without data augmentation:
"""
history=model.fit([x_train, y_train], [y_train, x_train], batch_size=128, epochs=13,
              validation_data=[[x_test_d, y_test_d], [y_test_d, x_test_d]])
    
  
"""

   # -----------------------------------Begin: Training with data augmentation -----------------------------------#
def train_generator(x, y, batch_size, shift_fraction):
        train_datagen = ImageDataGenerator(width_shift_range=shift_fraction,
                                           height_shift_range=shift_fraction,
                                           rotation_range=15,
                                           fill_mode='nearest')  # shift up to 2 pixel for MNIST
        generator = train_datagen.flow(x, y, batch_size=batch_size)
        while 1:
            x_batch, y_batch = generator.next()
            yield ([x_batch, y_batch], [y_batch, x_batch])
        
      # Training with data augmentation. If shift_fraction=0., also no augmentation.
history=model.fit_generator(generator=train_generator(x_train, y_train, 128, 0.2),
                        steps_per_epoch=int(1*y_train.shape[0] /128),
                        epochs=20,
                        validation_data=[[x_test, y_test], [y_test, x_test]]
                        )
    # -----------------------------------End: Training with data augmentation -----------------------------------#

In [ ]:
model.save('Caps_Massey.h5')

/content/drive/My Drive/AI


In [ ]:
score = model.evaluate([x_test, y_test], [y_test, x_test], verbose=0)
print('Test accuracy:', score[3])
print('Test loss:', score[1])

In [ ]:
%%time
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=90)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    

# Compute confusion matrix
class_names = ['class 1(A)', 'class 2(B)', 'class 3(C)','class 4(D)','class 5(E)','class 6(F)','class 7(G)','class 8(H)','class 9(I)','class 10(J)','class 11(K)','class 12(L)','class 13(M)','class 14(N)','class 15(O)','class 16(P)','class 17(Q)','class 18(R)','class 19(S)','class 20(T)','class 21(U)','class 22(V)','class 23(W)','class 24(X)','class 25(Y)','class 26(Z)']
y_pred,xrec = model.predict([x_test, y_test])
y_pred=np.argmax(y_pred, 1)
cnf_matrix = confusion_matrix(np.argmax([y_test, x_test][0],axis=1), y_pred)
np.set_printoptions(precision=2)
h=len(xrec)
print(cnf_matrix.shape)
class_names = ["A", "B", "C", "D", "E", "F", "G", "H", "I","J","K","L","M","N","O","P","Q","R","S","T","U","V","W","X","Y","Z"]
plt.figure()
fig = plt.gcf()
fig.set_size_inches(10, 7)
plot_confusion_matrix(cnf_matrix, classes=class_names,
                      title='CapsNet_1, without normalization',
                      cmap="Blues")
plt.savefig('CapsNet_1_confusion_matrix.png')
target_names = ['class 1(A)', 'class 2(B)', 'class 3(C)','class 4(D)','class 5(E)','class 6(F)','class 7(G)','class 8(H)','class 9(I)','class 10(J)','class 11(K)','class 12(L)','class 13(M)','class 14(N)','class 15(O)','class 16(P)','class 17(Q)','class 18(R)','class 19(S)','class 20(T)','class 21(U)','class 22(V)','class 23(W)','class 24(X)','class 25(Y)','class 26(Z)']

print(classification_report(np.argmax([y_test,x_test][0],axis=1), y_pred,target_names=target_names))

plt.show()

In [ ]:
#-----------------------------------------------CNN_Model------------------------------------------------------------------------------
model = Sequential()
model.add(Conv2D(filters = 16,kernel_size = (5,5),input_shape = (28,28,1),activation = 'relu',padding = 'same'))
model.add(MaxPooling2D((2,2)))
model.add(Conv2D(filters = 32,kernel_size = (5,5),padding = 'same',activation = 'relu'))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(32,kernel_size = (5,5),padding = 'same',activation = 'relu'))
model.add(Dropout(0.75))
model.add(Flatten())
model.add(Dense(64,activation = 'relu'))
model.add(Dropout(0.25))
model.add(Dense(26,activation = 'softmax'))
model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
history=model.fit(x_train, y_train,
          batch_size=128,
          epochs=15,
          verbose=1,
          validation_split=0.25)


In [ ]:
model.save('CNN_Massey.h5') 

In [ ]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
# with data augmentation:
model.compile(loss='categorical_crossentropy', optimizer=optimizers.Adam(lr=0.0009), metrics=['accuracy'])

gen = ImageDataGenerator(width_shift_range=0.2,
                         height_shift_range=0.2,
                         rotation_range=15,
                         fill_mode='nearest')

test_gen = ImageDataGenerator()

train_generator = gen.flow(x_train, y_train, batch_size=128)
test_generator = test_gen.flow(x_test, y_test, batch_size=128)

history=model.fit_generator(train_generator, steps_per_epoch=1*y_train.shape[0]/128, epochs=15, 
                    validation_data=test_generator, validation_steps=1*test.shape[0]/128)


In [ ]:
model.save('CNN_with_DA_Massey.h5') 

/content/drive/My Drive/AI


In [ ]:
score = model.evaluate(x_test, y_test, verbose=10)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=90)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    

# Compute confusion matrix
class_names = ['class 1(A)', 'class 2(B)', 'class 3(C)','class 4(D)','class 5(E)','class 6(F)','class 7(G)','class 8(H)','class 9(I)','class 11(K)','class 12(L)','class 13(M)','class 14(N)','class 15(O)','class 16(P)','class 17(Q)','class 18(R)','class 19(S)','class 20(T)','class 21(U)','class 22(V)','class 23(W)','class 24(X)','class 25(Y)']
y_pred = model.predict(x_test)
y_pred=np.argmax(y_pred, 1)
cnf_matrix = confusion_matrix(np.argmax(y_test,axis=1), y_pred)
print('arg:',np.argmax(y_test,axis=1))
np.set_printoptions(precision=2)

class_names = ["A", "B", "C", "D", "E", "F", "G", "H", "I","K","L","M","N","O","P","Q","R","S","T","U","V","W","X","Y"]

plt.figure()
fig = plt.gcf()
fig.set_size_inches(10, 7)
plot_confusion_matrix(cnf_matrix, classes=class_names,
                      title='CNN, without normalization',
                      cmap="Blues")
target_names = ['class 1(A)', 'class 2(B)', 'class 3(C)','class 4(D)','class 5(E)','class 6(F)','class 7(G)','class 8(H)','class 9(I)','class 11(K)','class 12(L)','class 13(M)','class 14(N)','class 15(O)','class 16(P)','class 17(Q)','class 18(R)','class 19(S)','class 20(T)','class 21(U)','class 22(V)','class 23(W)','class 24(X)','class 25(Y)']
print(classification_report(np.argmax([y_test][0],axis=1), y_pred,target_names=target_names))
plt.show()

In [ ]:
custom_objects={'CapsuleLayer': CapsuleLayer,'Mask':Mask,'Length':Length,'margin_loss':margin_loss}
model = load_model('Caps_with_DA_Massey.h5', custom_objects=custom_objects)
score = model.evaluate([x_test, y_test], [y_test, x_test], verbose=0)
print('Test accuracy:', score[3])
print('Test loss:', score[1])
layer_name = 'digitcaps'

intermediate_layer_model = Model(inputs=model.input,
                                 outputs=model.get_layer(layer_name).output)
Z1_train_DA = intermediate_layer_model.predict([x_train, y_train])
Z1_test_DA = intermediate_layer_model.predict([x_test, y_test])
print(Z1_train_DA.shape)
print(Z1_test_DA.shape)

Z1_train_DA=Z1_train_DA.reshape(27455, 400)
Z1_test_DA=Z1_test_DA.reshape(7172, 400)
print(Z1_train_DA.shape)
print(Z1_test_DA.shape)

In [ ]:
model = load_model('CNN_with_DA_Massey.h5')  # the original model
model.summary()
score = model.evaluate(x_test, y_test, verbose=10)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
layer_name = 'flatten'
intermediate_layer_model = Model(inputs=model.input,
                                 outputs=model.get_layer(layer_name).output)
Z2_train_DA = intermediate_layer_model.predict(x_train)
Z2_test_DA = intermediate_layer_model.predict(x_test)
print(Z2_train_DA.shape)
print(Z2_test_DA.shape)

In [ ]:
Z_train=np.concatenate((Z1_train_DA, Z2_train_DA), axis=1)
Z_test=np.concatenate((Z1_test_DA, Z2_test_DA), axis=1)
print(Z_train.shape)
print(Z_test.shape)

In [ ]:
scaler = StandardScaler(N)# Z_train.shape=(N,M)
scaler.fit(Z_train)
X_sc_train = scaler.transform(Z_train)
X_sc_test = scaler.transform(Z_test)

In [ ]:
pca = PCA(n_components=N)# Z_train.shape=(N,M)
pca.fit(Z_train)

plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('Number of components')
plt.ylabel('Cumulative explained variance')

In [ ]:
NCOMPONENTS = 1000

pca = PCA(n_components=NCOMPONENTS)
X_pca_train = pca.fit_transform(X_sc_train)
X_pca_test = pca.transform(X_sc_test)
pca_std = np.std(X_pca_train)

print(X_sc_train.shape)
print(X_pca_train.shape)
print(X_pca_test.shape)

In [ ]:
poly = svm.SVC(kernel='linear').fit(X_pca_train, np.argmax(y_train,axis=1))
poly_pred = poly.predict(X_pca_test)

poly_accuracy = accuracy_score(np.argmax(y_test_d,axis=1), poly_pred)
poly_f1 = f1_score(np.argmax(y_test_d,axis=1), poly_pred, average='weighted')
print('Accuracy (Polynomial Kernel): ', "%.2f" % (poly_accuracy*100))
print('F1 (Polynomial Kernel): ', "%.2f" % (poly_f1*100))

In [ ]:
%%time
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=90)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    

# Compute confusion matrix
class_names = ['class 1(A)', 'class 2(B)', 'class 3(C)','class 4(D)','class 5(E)','class 6(F)','class 7(G)','class 8(H)','class 9(I)','class 10(J)','class 11(K)','class 12(L)','class 13(M)','class 14(N)','class 15(O)','class 16(P)','class 17(Q)','class 18(R)','class 19(S)','class 20(T)','class 21(U)','class 22(V)','class 23(W)','class 24(X)','class 25(Y)','class 26(Z)']
y_pred = poly.predict(X_pca_test)
#y_pred=np.argmax(y_pred, 1)
cnf_matrix = confusion_matrix(np.argmax(y_test_d,axis=1), y_pred)
print('arg:',np.argmax(y_test_d,axis=1))
np.set_printoptions(precision=2)

class_names = ["A", "B", "C", "D", "E", "F", "G", "H", "I","J","K","L","M","N","O","P","Q","R","S","T","U","V","W","X","Y","Z"]

plt.figure()
fig = plt.gcf()
fig.set_size_inches(10, 7)
plot_confusion_matrix(cnf_matrix, classes=class_names,
                      title='CNN, without normalization',
                      cmap="Blues")
target_names =['class 1(A)', 'class 2(B)', 'class 3(C)','class 4(D)','class 5(E)','class 6(F)','class 7(G)','class 8(H)','class 9(I)','class 10(J)','class 11(K)','class 12(L)','class 13(M)','class 14(N)','class 15(O)','class 16(P)','class 17(Q)','class 18(R)','class 19(S)','class 20(T)','class 21(U)','class 22(V)','class 23(W)','class 24(X)','class 25(Y)','class 26(Z)']
print(classification_report(np.argmax([y_test_d][0],axis=1), y_pred,target_names=target_names))
plt.show()